In [2]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [3]:
from ktrain import text
import os

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Load dataset
file_path = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/Social_plus_batch_1_batch_2_merged.xlsx"
social_df = pd.read_excel(file_path)


In [6]:
# Perform train-test split (keeping stratification)
train_social, test_social = train_test_split(
    social_df, test_size=0.22, random_state=42, stratify=social_df["label"]
)

# Verify counts
print(f"social Train: {len(train_social)}, Test: {len(test_social)}, Total: {len(social_df)}")

# Count total samples
total_train = len(train_social)
total_test = len(test_social)

# Count positive (Class 1) samples
train_positives = train_social["label"].sum()
test_positives = test_social["label"].sum()

# Calculate percentages
train_positive_pct = (train_positives / total_train) * 100
test_positive_pct = (test_positives / total_test) * 100

# Print results
print(f"📊 social Dataset Train-Test Split Results:")
print(f"✅ Train: {total_train} samples (Positives: {train_positives}, {train_positive_pct:.2f}%)")
print(f"✅ Test: {total_test} samples (Positives: {test_positives}, {test_positive_pct:.2f}%)")
print(f"🔹 Total Dataset: {len(social_df)} samples")


# Define save directory
save_dir = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/soc_train_test"
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Save train and test sets
train_social.to_excel(f"{save_dir}/train_social.xlsx", index=False)
test_social.to_excel(f"{save_dir}/test_social.xlsx", index=False)

print("✅ social train and test sets successfully created and saved!")

social Train: 3043, Test: 859, Total: 3902
📊 social Dataset Train-Test Split Results:
✅ Train: 3043 samples (Positives: 685, 22.51%)
✅ Test: 859 samples (Positives: 193, 22.47%)
🔹 Total Dataset: 3902 samples
✅ social train and test sets successfully created and saved!


In [7]:
MAXLEN = 150

In [8]:
X = train_social['sentence']
y = train_social['label']

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [10]:
# Define model name
model_name = 'microsoft/deberta-v3-base'  # You can also use 'microsoft/deberta-v3-small' for a smaller model

# Initialize DeBERTa transformer
deberta_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0, 1])

# Preprocess training and validation sets
training_set = deberta_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = deberta_transformer.preprocess_test(X_test.tolist(), y_test.tolist())

# Get DeBERTa classifier model
deberta_base_model = deberta_transformer.get_classifier()

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/736M [00:00<?, ?B/s]

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 56


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/Users/gbaldonado/miniforge3/envs/ml_env/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 37
	99percentile : 59


In [11]:
deberta_learner = ktrain.get_learner(deberta_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [12]:
deberta_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
406/406 [==============================] - 1130s 3s/step - loss: 0.5483 - accuracy: 0.7494 - val_loss: 0.5116 - val_accuracy: 0.7750
Epoch 2/5
406/406 [==============================] - 950s 2s/step - loss: 0.3999 - accuracy: 0.8357 - val_loss: 0.3103 - val_accuracy: 0.8719
Epoch 3/5
406/406 [==============================] - 915s 2s/step - loss: 0.2856 - accuracy: 0.8850 - val_loss: 0.3027 - val_accuracy: 0.8818
Epoch 4/5
406/406 [==============================] - 969s 2s/step - loss: 0.2656 - accuracy: 0.8998 - val_loss: 0.3027 - val_accuracy: 0.8736
Epoch 5/5
406/406 [==============================] - 918s 2s/step - loss: 0.2170 - accuracy: 0.9154 - val_loss: 0.3494 - val_accuracy: 0.8686


In [15]:
deberta_learner.validate(class_names=deberta_transformer.get_classes())

20/20 [==============================] - 18s 884ms/step
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       472
           1       0.68      0.80      0.73       137

    accuracy                           0.87       609
   macro avg       0.81      0.84      0.82       609
weighted avg       0.88      0.87      0.87       609



array([[420,  52],
       [ 28, 109]])

In [16]:
deberta_learner.model.summary()

Model: "tf_deberta_v2_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 deberta (TFDebertaV2MainLa  multiple                  183831552 
 yer)                                                            
                                                                 
 pooler (TFDebertaV2Context  multiple                  590592    
 Pooler)                                                         
                                                                 
 cls_dropout (TFDebertaV2St  multiple                  0         
 ableDropout)                                                    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 184423682 (703.52 MB)
Trainable params: 184423682 (703.52 MB)
Non-trainable

In [17]:
deberta_predictor = ktrain.get_predictor(deberta_learner.model, preproc=deberta_transformer)

In [18]:
deberta_test_data = test_social['sentence'].tolist()
deberta_test_label = test_social['label'].tolist()

In [19]:
y_pred_deberta = deberta_predictor.predict(deberta_test_data)

In [20]:
y_pred_deberta = [int(x) for x in y_pred_deberta]

In [21]:
tn, fp, fn, tp = confusion_matrix(deberta_test_label, y_pred_deberta).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 600, False Positive: 66, False Negative: 50, True Positive: 143


In [22]:
print('  Classification Report:\n',classification_report(deberta_test_label,y_pred_deberta),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.90      0.91       666
           1       0.68      0.74      0.71       193

    accuracy                           0.86       859
   macro avg       0.80      0.82      0.81       859
weighted avg       0.87      0.86      0.87       859
 



In [25]:
print('  Classification Report:\n',classification_report(deberta_test_label,y_pred_deberta),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.92      0.95       117
           1       0.72      0.92      0.81        25

    accuracy                           0.92       142
   macro avg       0.85      0.92      0.88       142
weighted avg       0.94      0.92      0.93       142
 



In [24]:
#deberta_predictor.save('./model/distilbert_base_uncased_model') # 256 MB

In [23]:
print("AUC roc score for deberta model: ", roc_auc_score(deberta_test_label,y_pred_deberta))

AUC roc score for deberta model:  0.8209167716939738
